In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandasql as ps
from pandasql import sqldf

In [2]:
basic_url = "https://fbref.com/en/comps/Big5/keepers/players/Big-5-European-Leagues-Stats"
advanced_url = "https://fbref.com/en/comps/Big5/keepersadv/players/Big-5-European-Leagues-Stats"

In [3]:
response1 = requests.get(basic_url).text.replace('<!--', '').replace('-->', '')
response2 = requests.get(advanced_url).text.replace('<!--', '').replace('-->', '')

dfb = pd.read_html(response1, header=1)[1]
dfa = pd.read_html(response2, header=1)[1]

pysqldf = lambda q: sqldf(q, globals())

q = """
SELECT *
FROM dfb
WHERE CAST(`Rk` as text) != 'Rk'
"""

dfb = pysqldf(q)

q = """
SELECT *
FROM dfa
WHERE CAST(`Rk` as text) != 'Rk'
"""

dfa = pysqldf(q)

In [4]:
dfa['PSxG/SoT'] = dfa['PSxG/SoT'].astype(float)
dfb['exs%'] = 0
dfb['exs%'] = dfb['exs%'].astype(float)

for i in range(dfb.shape[0]):
    ex = dfa.iloc[i, dfa.columns.get_loc('PSxG/SoT')]
    dfb.iloc[i, dfb.columns.get_loc('exs%')] = 100 * (1 - ex)

In [5]:
dfb['GA-xGA'] = np.nan

dfb['Save%'] = dfb['Save%'].astype(float)

q = """
SELECT *, 
(CASE 
    WHEN Comp IN ('de Bundesliga', 'eng Premier League') THEN 2.7
    WHEN Comp = 'fr Ligue 1' THEN 1.3
    WHEN Comp = 'es La Liga' THEN 0.6
    WHEN Comp = 'it Serie A' THEN 2.2
    ELSE 0
END + [Save%] - [exs%]) AS [GA-xGA]
FROM dfb
"""

dfb = pysqldf(q)
dfb['GA-xGA'] = dfb['GA-xGA'].round(4)

In [6]:
dfb

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,CS,CS%,PKatt,PKA,PKsv,PKm,Save%.1,Matches,exs%,GA-xGA
0,1,Álvaro Aceves,es ESP,GK,Valladolid,es La Liga,19-278,2003,1,0,...,0,None,0,0,0,0,None,Matches,96.0,4.6
1,2,Julen Agirrezabala,es ESP,GK,Athletic Club,es La Liga,22-125,2000,6,5,...,2,40.0,0,0,0,0,None,Matches,67.0,12.2
2,3,Doğan Alemdar,tr TUR,GK,Rennes,fr Ligue 1,20-183,2002,4,3,...,0,0.0,1,0,1,0,100.0,Matches,60.0,-22.3
3,4,Alisson,br BRA,GK,Liverpool,eng Premier League,30-210,1992,32,32,...,11,34.4,3,2,1,0,33.3,Matches,66.0,8.2
4,5,Alphonse Areola,fr FRA,GK,West Ham,eng Premier League,30-062,1993,5,2,...,1,50.0,2,2,0,0,0.0,Matches,67.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,182,Joseph Whitworth,eng ENG,GK,Crystal Palace,eng Premier League,19-060,2004,2,2,...,0,0.0,0,0,0,0,None,Matches,77.0,-24.3
182,183,Robin Zentner,de GER,GK,Mainz 05,de Bundesliga,28-184,1994,23,23,...,7,30.4,3,1,1,1,50.0,Matches,71.0,-0.9
183,184,Michael Zetterer,de GER,GK,Werder Bremen,de Bundesliga,27-292,1995,2,1,...,0,0.0,0,0,0,0,None,Matches,76.0,5.3
184,185,Jeroen Zoet,nl NED,GK,Spezia,it Serie A,32-114,1991,4,3,...,0,0.0,0,0,0,0,None,Matches,81.0,-3.8


In [7]:
q = """
SELECT *
FROM dfb
ORDER BY `GA-xGA` DESC
"""

dfb = pysqldf(q)

In [8]:
q = """
SELECT *
FROM dfb
WHERE CAST(MP as float) >= 10
"""

dfb = pysqldf(q)

In [9]:
query = """
    SELECT Player, [GA-xGA] 
    FROM dfb 
    LIMIT 20
"""
result = ps.sqldf(query, locals())

result

,Player,GA-xGA
0,Manuel Neuer,11.2
1,Frederik Rønnow,11.2
2,Mattia Perin,10.4
3,Benjamin Lecomte,9.7
4,Marc-André ter Stegen,8.3
5,Alisson,8.2
6,Yann Sommer,7.7
7,Yehvann Diouf,7.6
8,Bernd Leno,7.1
9,Pau López,6.7


In [10]:
query = """
    SELECT Player, [GA-xGA] 
    FROM dfb 
    ORDER BY ROWID DESC 
    LIMIT 20
"""
result = ps.sqldf(query, locals())

result

,Player,GA-xGA
0,Gavin Bazunu,-15.1
1,Agustín Marchesín,-12.9
2,Mark Travers,-11.6
3,Andrea Consigli,-9.5
4,Benjamin Lecomte,-8.4
5,Fernando Pacheco,-7.9
6,Sergio Asenjo,-7.6
7,Marvin Schwäbe,-7.2
8,Benjamin Leroy,-6.9
9,Oliver Christensen,-6.8


In [11]:
query = """
    SELECT Player, [GA-xGA] 
    FROM dfb 
    ORDER BY ROWID ASC 
"""
result = ps.sqldf(query, locals())

for i in range(len(result)):
    print(str(i+1) + ": " + result.iloc[i]['Player'] + ", " + str(result.iloc[i]['GA-xGA']))

1: Manuel Neuer, 11.2
2: Frederik Rønnow, 11.2
3: Mattia Perin, 10.4
4: Benjamin Lecomte, 9.7
5: Marc-André ter Stegen, 8.3
6: Alisson, 8.2
7: Yann Sommer, 7.7
8: Yehvann Diouf, 7.6
9: Bernd Leno, 7.1
10: Pau López, 6.7
11: Ivan Provedel, 6.6
12: Gerónimo Rulli, 5.7
13: Ralf Fährmann, 5.5
14: Paulo Gazzaniga, 5.5
15: Guillermo Ochoa, 5.3
16: Mory Diaw, 5.1
17: Gregor Kobel, 4.8
18: Brice Samba, 4.8
19: Jordan Pickford, 4.7
20: Mark Flekken, 4.6
21: Gianluigi Donnarumma, 4.3
22: Jordi Masip, 4.1
23: Anthony Lopes, 4.0
24: Michele Di Gregorio, 3.9
25: Neto, 3.8
26: Kepa Arrizabalaga, 3.7
27: Steve Mandanda, 3.6
28: Jan Oblak, 3.5
29: Nick Pope, 3.2
30: David Raya, 3.2
31: Alban Lafont, 3.1
32: Łukasz Skorupski, 3.0
33: Vito Mannone, 2.9
34: Luigi Sepe, 2.8
35: Pepe Reina, 2.6
36: Alex Meret, 2.5
37: Ionuț Radu, 2.5
38: Guglielmo Vicario, 2.3
39: Keylor Navas, 1.9
40: Jonas Omlin, 1.9
41: Édgar Badía, 1.8
42: Wladimiro Falcone, 1.8
43: Fernando, 1.8
44: Rui Silva, 1.8
45: Marco Bizot, 1.7

In [13]:
print(np.mean(dfb['GA-xGA']))
print(np.var(dfb['GA-xGA']))
print(np.std(dfb['GA-xGA']))

-0.0924369747899159
23.817169691405972
4.880283771606521
